We attempt to analysis each reviews by extracting the problems/benefits of the application mentioned in each reviews
To do so, we utilise the hugging face question-answering pipeline
Similarly, hugging face provides us with the sentiment analysis pipeline, allowing us to check the sentiment value of each review

In [3]:
#libraries
import pandas as pd


In [4]:
#import data
appStore = pd.read_csv('AppStoreData.csv')
googlePlay = pd.read_csv('PlayStoreData.csv')

In [81]:
#combine review data 
as_review = appStore['review']
gp_review = googlePlay['text']

reviews = as_review.tolist() + gp_review.tolist()

In [83]:
#data cleaning to remove weird comments
print(reviews)

['Great banking app with attractive interest rates! Please allow us to add and/or save payees so we don’t have to keep typing out UEN numbers or account numbers. Would be nice to be able to add the debit card to Apple Pay too!!', 'A bank like no other, no bank have such amazing feature to separate your money into different “pockets” to keep track of your goals. Moreover the interest rate they give is the highest for the no effort! Interest are being deposit daily and it’s amazing to see 💵 coming in to the bank daily! \n\nThe user interfaces is so intuitive that every button can be found with your eyes close, that makes money transfer so much smoother. Also the colours is so slick that I want to turn the app on everyday just to look at it and see the daily interest\n\nLots of potential for the product and I can’t wait to see more of it.', 'Notice that the drop in interest rate of 0.8% in saving accounts. GXS just come out with a reason to say many consumers want a increase in interest r

In [84]:
from transformers import pipeline
sent_pipeline = pipeline("sentiment-analysis")
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Sentiment analysis

In [85]:
sentiment_scoring = [sent_pipeline(review) for review in reviews]

In [86]:
label = []
score = []
for lst in sentiment_scoring:
    label.append(lst[0]['label'])
    score.append((lst[0]['score']))

In [87]:
sentiment_df = pd.DataFrame(list(zip(reviews, label, score)), columns=['review','label', 'sentiment_score'])

In [88]:
sentiment_df.head()

,review,label,sentiment_score
0,Great banking app with attractive interest rat...,POSITIVE,0.994833
1,"A bank like no other, no bank have such amazin...",POSITIVE,0.999390
2,Notice that the drop in interest rate of 0.8% ...,NEGATIVE,0.998625
3,Sending money into my GXS account is a breeze ...,NEGATIVE,0.994006
4,I have to say that the UI/UX is one of the bes...,POSITIVE,0.998633


Review analysis

In [92]:
reviews[0]

'Great banking app with attractive interest rates! Please allow us to add and/or save payees so we don’t have to keep typing out UEN numbers or account numbers. Would be nice to be able to add the debit card to Apple Pay too!!'

In [151]:
#context = reviews #this part, make it into a forloop or a function to output the desired data

def report(review):
    #should return a dictionary i guess
    ans = dict.fromkeys(['Good', 'Suggested improvements'], [])
    ans['Good'] = qa_pipeline(question="What is good and positive about this application?", context=review)["answer"]
    ans['Suggested improvements'] = qa_pipeline(question="How can the pplication improve?", context=review)["answer"]
    return ans

context = reviews[0]
print(qa_pipeline(question="What is good and positive about this application?", context=context))
print(qa_pipeline(question="What can be added to the application?", context=context))
print(qa_pipeline(question="What do the application lack", context=context))

{'score': 0.27957335114479065, 'start': 23, 'end': 48, 'answer': 'attractive interest rates'}
{'score': 0.7127752304077148, 'start': 196, 'end': 206, 'answer': 'debit card'}
{'score': 0.10196660459041595, 'start': 128, 'end': 158, 'answer': 'UEN numbers or account numbers'}


In [150]:
check = reviews[0]
print(check)
report(check)

Great banking app with attractive interest rates! Please allow us to add and/or save payees so we don’t have to keep typing out UEN numbers or account numbers. Would be nice to be able to add the debit card to Apple Pay too!!


{'Good': 'attractive interest rates',
 'Suggested improvements': 'Please allow us to add and/or save payees'}

In [107]:
#qa on all the reviews
idk = [report(review) for review in reviews]

In [108]:
print(idk)

[{'Good': ['attractive interest rates', 'debit card'], 'Suggested improvements': ['attractive interest rates', 'debit card']}, {'Good': ['user interfaces', 'interest'], 'Suggested improvements': ['user interfaces', 'interest']}, {'Good': ['the drop in interest rate of 0.8% in saving accounts', 'GXS can increase the main account interest'], 'Suggested improvements': ['the drop in interest rate of 0.8% in saving accounts', 'GXS can increase the main account interest']}, {'Good': ['it’s annoying that the advice provided', 'money'], 'Suggested improvements': ['it’s annoying that the advice provided', 'money']}, {'Good': ['customer experience and design', 'asap can’t wait to see what you have to offer'], 'Suggested improvements': ['customer experience and design', 'asap can’t wait to see what you have to offer']}, {'Good': ['believe trust bank Singapore', 'excuses'], 'Suggested improvements': ['believe trust bank Singapore', 'excuses']}, {'Good': ['high interest rates', 'high interest rates

Other question answering method that may be more accuracte

In [129]:

# import
from transformers.pipelines import pipeline
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer

# var
model_name = "deepset/xlm-roberta-base-squad2"

# generate pipeline
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

input = {
    'question': 'How can the application improve?' ,
    'context': 'My name is Mohit. I am going to visit my grandmother. She is old.'
}
print(nlp(input))
## Output --> {'score': 0.30, 'start': 10, 'end': 17, 'answer': ' Mohit.'}


config.json: 100%|██████████| 605/605 [00:00<00:00, 2.81MB/s]
model.safetensors: 100%|██████████| 1.11G/1.11G [01:09<00:00, 15.9MB/s]
tokenizer_config.json: 100%|██████████| 79.0/79.0 [00:00<00:00, 37.2kB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:01<00:00, 2.72MB/s]
special_tokens_map.json: 100%|██████████| 150/150 [00:00<00:00, 815kB/s]


{'score': 0.3041093349456787, 'start': 10, 'end': 17, 'answer': ' Mohit.'}


Comparing hugging face 1 and nlp

In [165]:
#i think what we can do is to analyse the sentiment of the review. If it is bad
print(reviews[3])
input = {
    'question': 'How can the application be improved?',
    'context': reviews[3]
}
print(nlp(input))

input = {
    'question': 'what is good about the application',
    'context': reviews[3]
}
print(nlp(input))

#compared to hugging
check = reviews[3]
report(check)


Sending money into my GXS account is a breeze and instantaneous - regardless of the amounts. I’m able to immediately see that my funds are in GXS. 

Transferring money OUT is a huge issue. Since June I’ve had problems transferring amounts higher than $500 back to my other banking accounts, each time a red banner will pop up and said something went wrong please try again later. TODAY I can’t transfer more than $1000 back to myself - even the $1000 had to be transferred in TWO transactions of $500 each. Customer service officers did their best to help each time but it’s annoying that the advice provided (killing the app, re-logging in with SingPass) still don’t work.
{'score': 0.14726075530052185, 'start': 608, 'end': 655, 'answer': ' (killing the app, re-logging in with SingPass)'}
{'score': 0.018554577603936195, 'start': 608, 'end': 655, 'answer': ' (killing the app, re-logging in with SingPass)'}


{'Good': 'it’s annoying that the advice provided',
 'Suggested improvements': 'still don’t work'}

In [1]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
